## TODO
- Brett: invertible reshaping (spatial/temporal)
- Brett: refactor model code into Python library
- Josh: simple PCA clustering

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('poster')
sns.set_style("whitegrid", {'axes.grid': False})
import h5py
import tensorflow as tf

from keras import backend as K
gpu_opts = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.4))
K.set_session(tf.Session(config=gpu_opts))
#K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
from autoencoder import load_data

#X = load_data('data/cleaned_data.mat', resample_size=256)
X = np.load('data/X_256.npy')

In [ ]:
%%time
import keras

model = keras.models.load_model('log/lstm128_x1_emb008_3m04_drop0_batch1024/weights.h5')

encode_model = keras.models.Model(inputs=model.input,
                                  outputs=[l for l in model.layers
                                           if isinstance(l, keras.layers.core.Dense)][0].output)
#inds = np.arange(20, X.shape[0], 40)
inds = np.arange(X.shape[0])
#encoding = encode_model.predict(X[inds])
encoding = np.load('log/lstm128_x1_emb008_3m04_drop0_batch1024/encoding.npy')

In [ ]:
%%time
from sklearn.decomposition import PCA

pca_model = PCA(16)
pca_model.fit(X.squeeze())

In [ ]:
X_fft = np.fft.rfft(X.squeeze())
X_fft[:, 5:-4] = 0.  # x_fft[:, 0:5] (both real and imaginary) are the features
X_fft_re = np.fft.irfft(X_fft)  # reconstruction

In [ ]:
#i = np.random.randint(0, X.shape[0])
#i = 20

plt.plot(X[i])
plt.plot(model.predict(X[[i]])[0])
plt.plot(pca_model.inverse_transform(pca_model.transform(X[[i]].squeeze())[0]))
plt.plot(X_fft_re[i])
mse_rnn = np.mean((X[i] - model.predict(X[[i]])[0]) ** 2)
mse_pca = np.mean((X[i].squeeze() - pca_model.inverse_transform(pca_model.transform(X[[i]].squeeze())[0])) ** 2)
mse_fft = np.mean((X[i].squeeze() - X_fft_re[i]) ** 2)
#plt.plot(X[i] - model.predict(X[[i]])[0])
plt.legend(['Data', 'RNN', 'PCA', 'FFT'])
plt.title(f"i={i}; MSE (RNN)={mse_rnn:1.5f}; MSE (PCA)={mse_pca:1.5f}; MSE (FFT)={mse_fft:1.5f}");

In [ ]:
loadings = pca_model.transform(X.squeeze())[inds]
fig, ax = plt.subplots(8, 2, figsize=(8, 36))
for j in range(encoding.shape[1]):
    ax[j, 0].imshow(encoding[:, j].reshape((256, 256)), cmap='viridis')#, vmin=encoding.min(), vmax=encoding.max())
    ax[j, 1].imshow(loadings[:, j].reshape((256, 256)), cmap='viridis')#, vmin=encoding.min(), vmax=encoding.max())

In [ ]:
e20 = encoding[20]
Y20 = model.predict(X[[20]])
plt.plot(decode_model.predict(e20[np.newaxis, :]).squeeze())
plt.plot(Y20.squeeze() + 1e-4)

In [ ]:
import scipy.io
import scipy.signal

VoltageData = scipy.io.loadmat('voltage_vector.mat')['BE_wave'].squeeze()
V = scipy.signal.resample(VoltageData, 40 * 128 * 2)
plt.plot(V[:256])

In [ ]:
i = np.argmin(np.sum(np.abs(encoding), axis=1))
encoding[i]

In [ ]:
#x = np.zeros((1, 8))
#x[0] = 0.
plt.plot(V[:256], decode_model.predict(encoding[i:i+1]).squeeze())
plt.plot(V[:256], X[i].squeeze())

In [ ]:
def model_to_decoder(model, layer=keras.layers.LSTM, size=128, embedding=8, num_layers=1):
    if num_layers > 1:
        raise NotImplementedError("TODO >1 layer")
        
    decode_model = keras.models.Sequential()
    decode_model.add(keras.layers.RepeatVector(X.shape[1], input_shape=(embedding,)))
    decode_model.add(keras.layers.Bidirectional(layer(size, return_sequences=True)))
    decode_model.add(keras.layers.TimeDistributed(keras.layers.Dense(1, activation='linear')))
    decode_model.set_weights(model.get_weights()[-len(decode_model.get_weights()):])

    return decode_model

In [3]:
from autoencoder import conv_auto
from keras.optimizers import Adam

model = conv_auto(32, 4, 8, n_step=X.shape[1],
                  kernel_size=3, drop_frac=0.)
model.compile(Adam(1e-4), loss='mse')

history = model.fit(X, X, epochs=1, batch_size=64,
                    callbacks=[TQDMCallback(),
#                               TensorBoard(log_dir=log_dir, write_graph=False),
#                               ModelCheckpoint(weights_path)],
                              ],
                    verbose=False)

TypeError: add() got an unexpected keyword argument 'input_shape'

In [ ]:
np.save('log/lstm128_x1_emb008_3m04_drop0_batch1024/encoding.npy', encoding, allow_pickle=False)